# Feature Engineering

In [1]:
#Import Libraries

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math
from geopy.distance import geodesic
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns
from sklearn.feature_selection import SelectKBest
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures



In [2]:
#Read CSV File
train_df = pd.read_csv('../cleaned_dataset/train.csv')
test_df = pd.read_csv('../cleaned_dataset/test.csv')
weather_df = pd.read_csv('../cleaned_dataset/weather.csv')
spray_df = pd.read_csv('../cleaned_dataset/spray.csv')

In [3]:
train_df

,block,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,tmax_stn1,tmin_stn1,tavg_stn1,depart_stn1,...,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903,species_others,street_others,trap_others
0,41,41.954690,-87.800991,9,1,0,88,60,74,10,...,0,0,0,0,0,0,0,0,0,0
1,41,41.954690,-87.800991,9,1,0,88,60,74,10,...,0,0,0,0,0,0,0,0,0,0
2,62,41.994991,-87.769279,9,1,0,88,60,74,10,...,0,0,0,0,0,0,0,0,0,0
3,79,41.974089,-87.824812,8,1,0,88,60,74,10,...,0,0,0,0,0,0,0,0,0,0
4,79,41.974089,-87.824812,8,4,0,88,60,74,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,51,41.763733,-87.742302,8,6,1,75,50,63,3,...,0,0,0,0,0,0,0,0,0,0
10502,58,41.987280,-87.666066,8,5,0,75,50,63,3,...,0,0,0,0,0,0,0,0,0,0
10503,17,41.912563,-87.668055,9,1,0,75,50,63,3,...,0,0,0,0,0,0,0,0,0,0
10504,71,42.009876,-87.807277,9,5,0,75,50,63,3,...,1,0,0,0,0,0,0,0,0,0


In [4]:
column_names = list(train_df.columns)
print(column_names)


['block', 'latitude', 'longitude', 'addressaccuracy', 'nummosquitos', 'wnvpresent', 'tmax_stn1', 'tmin_stn1', 'tavg_stn1', 'depart_stn1', 'dewpoint_stn1', 'wetbulb_stn1', 'heat_stn1', 'cool_stn1', 'sunrise_stn1', 'sunset_stn1', 'depth_stn1', 'snowfall_stn1', 'preciptotal_stn1', 'stnpressure_stn1', 'sealevel_stn1', 'resultspeed_stn1', 'resultdir_stn1', 'avgspeed_stn1', 'tmax_stn2', 'tmin_stn2', 'tavg_stn2', 'dewpoint_stn2', 'wetbulb_stn2', 'heat_stn2', 'cool_stn2', 'preciptotal_stn2', 'stnpressure_stn2', 'sealevel_stn2', 'resultspeed_stn2', 'resultdir_stn2', 'avgspeed_stn2', 'year', 'month', 'day', 'species_CULEX PIPIENS', 'species_CULEX PIPIENS/RESTUANS', 'species_CULEX RESTUANS', 'species_CULEX SALINARIUS', 'species_CULEX TARSALIS', 'species_CULEX TERRITANS', 'street_ E 105TH ST', 'street_ E 111TH ST', 'street_ E 115TH ST', 'street_ E 118TH ST', 'street_ E 130TH ST', 'street_ E 138TH ST', 'street_ E 67TH ST', 'street_ E 91ST PL', 'street_ E 91ST ST', 'street_ E RANDOLPH ST', 'street_ 

## Creating Interaction Features: 

In [5]:
# Calculate the duration of daylight
train_df['daylight_duration'] = train_df['sunset_stn1'] - train_df['sunrise_stn1']

# Print the updated DataFrmame
print(train_df.head())

   block   latitude  longitude  addressaccuracy  nummosquitos  wnvpresent  \
0     41  41.954690 -87.800991                9             1           0   
1     41  41.954690 -87.800991                9             1           0   
2     62  41.994991 -87.769279                9             1           0   
3     79  41.974089 -87.824812                8             1           0   
4     79  41.974089 -87.824812                8             4           0   

   tmax_stn1  tmin_stn1  tavg_stn1  depart_stn1  ...  trap_T235  trap_T236  \
0         88         60         74           10  ...          0          0   
1         88         60         74           10  ...          0          0   
2         88         60         74           10  ...          0          0   
3         88         60         74           10  ...          0          0   
4         88         60         74           10  ...          0          0   

   trap_T237  trap_T238  trap_T900  trap_T903  species_others  stree

In [6]:
# Convert temperatures from Fahrenheit to Celsius
train_df['tavg_stn2_celsius'] = (train_df['tavg_stn2'] - 32) * 5/9
train_df['tavg_stn1_celsius'] = (train_df['tavg_stn1'] - 32) * 5/9

# Calculate actual vapor pressure for each station
train_df['actual_vapor_pressure_stn2'] = 6.11 * 10**((7.5 * train_df['tavg_stn2_celsius']) / (237.7 + train_df['tavg_stn2_celsius']))
train_df['actual_vapor_pressure_stn1'] = 6.11 * 10**((7.5 * train_df['tavg_stn1_celsius']) / (237.7 + train_df['tavg_stn1_celsius']))

# Calculate saturation vapor pressure for each station
train_df['saturation_vapor_pressure_stn2'] = 6.11 * 10**((7.5 * train_df['dewpoint_stn2']) / (237.7 + train_df['dewpoint_stn2']))
train_df['saturation_vapor_pressure_stn1'] = 6.11 * 10**((7.5 * train_df['dewpoint_stn1']) / (237.7 + train_df['dewpoint_stn1']))

# Calculate relative humidity for each station
train_df['relative_humidity_stn2'] = (train_df['actual_vapor_pressure_stn2'] / train_df['saturation_vapor_pressure_stn2']) * 100
train_df['relative_humidity_stn1'] = (train_df['actual_vapor_pressure_stn1'] / train_df['saturation_vapor_pressure_stn1']) * 100


In [7]:
# Drop the specified columns
train_df.drop(['tavg_stn2', 'dewpoint_stn2', 'dewpoint_stn1', 'tavg_stn1', 'sunset_stn1', 'sunrise_stn1', 'actual_vapor_pressure_stn2','actual_vapor_pressure_stn1','saturation_vapor_pressure_stn2','saturation_vapor_pressure_stn1'], axis=1, inplace=True)


In [8]:
# Export the modified DataFrame to a new CSV file
train_df.to_csv('../cleaned_dataset/cleaned_fe_train.csv', index=False)

In [9]:
# Calculate the duration of daylight
test_df['daylight_duration'] = test_df['sunset_stn1'] - test_df['sunrise_stn1']

# Print the updated DataFrmame
print(test_df.head())

   block  latitude  longitude  addressaccuracy  tmax_stn1  tmin_stn1  \
0     41  41.95469 -87.800991                9         86         61   
1     41  41.95469 -87.800991                9         86         61   
2     41  41.95469 -87.800991                9         86         61   
3     41  41.95469 -87.800991                9         86         61   
4     41  41.95469 -87.800991                9         86         61   

   tavg_stn1  depart_stn1  dewpoint_stn1  wetbulb_stn1  ...  trap_T232  \
0         74            7             56            64  ...          0   
1         74            7             56            64  ...          0   
2         74            7             56            64  ...          0   
3         74            7             56            64  ...          0   
4         74            7             56            64  ...          0   

   trap_T233  trap_T235  trap_T236  trap_T237  trap_T238  trap_T900  \
0          0          0          0          0      

In [10]:
# Convert temperatures from Fahrenheit to Celsius
test_df['tavg_stn2_celsius'] = (test_df['tavg_stn2'] - 32) * 5/9
test_df['tavg_stn1_celsius'] = (test_df['tavg_stn1'] - 32) * 5/9

# Calculate actual vapor pressure for each station
test_df['actual_vapor_pressure_stn2'] = 6.11 * 10**((7.5 * test_df['tavg_stn2_celsius']) / (237.7 + test_df['tavg_stn2_celsius']))
test_df['actual_vapor_pressure_stn1'] = 6.11 * 10**((7.5 * test_df['tavg_stn1_celsius']) / (237.7 + test_df['tavg_stn1_celsius']))

# Calculate saturation vapor pressure for each station
test_df['saturation_vapor_pressure_stn2'] = 6.11 * 10**((7.5 * test_df['dewpoint_stn2']) / (237.7 + test_df['dewpoint_stn2']))
test_df['saturation_vapor_pressure_stn1'] = 6.11 * 10**((7.5 * test_df['dewpoint_stn1']) / (237.7 + test_df['dewpoint_stn1']))

# Calculate relative humidity for each station
test_df['relative_humidity_stn2'] = (test_df['actual_vapor_pressure_stn2'] / test_df['saturation_vapor_pressure_stn2']) * 100
test_df['relative_humidity_stn1'] = (test_df['actual_vapor_pressure_stn1'] / test_df['saturation_vapor_pressure_stn1']) * 100


In [11]:
# Drop the specified columns
test_df.drop(['tavg_stn2', 'dewpoint_stn2', 'dewpoint_stn1', 'tavg_stn1', 'sunset_stn1', 'sunrise_stn1', 'actual_vapor_pressure_stn2','actual_vapor_pressure_stn1','saturation_vapor_pressure_stn2','saturation_vapor_pressure_stn1'], axis=1, inplace=True)



In [12]:
# Export the modified DataFrame to a new CSV file
test_df.to_csv('../cleaned_dataset/cleaned_fe_test.csv', index=False)

In [13]:
# Check for null values in test_df
null_values = test_df.isnull().sum()
print(null_values)


block                     0
latitude                  0
longitude                 0
addressaccuracy           0
tmax_stn1                 0
                         ..
daylight_duration         0
tavg_stn2_celsius         0
tavg_stn1_celsius         0
relative_humidity_stn2    0
relative_humidity_stn1    0
Length: 308, dtype: int64


In [14]:
# Check for null values in test_df and identify columns with null values
null_columns = test_df.columns[test_df.isnull().any()]
print(null_columns)


Index([], dtype='object')
